## Imports

In [1]:
import os
from tqdm import tqdm
import glob
import sys
import time
import pickle
import cv2
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Load DataSet

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
%ls

drive/  sample_data/


In [5]:
%cd /content/drive/MyDrive/DL_Project

/content/drive/MyDrive/DL_Project


In [6]:
BASE_DIR = './Dataset/'

In [7]:
class CaptchaDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(f"{root_dir}{annotation_file}", dtype=str)
        self.annotations = self.annotations[self.annotations['length']=='3']
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index]
        img = Image.open(
            os.path.join( self.root_dir, img_id['category'], f"Len{img_id['length']}Char", img_id['file'])
          ).convert("L")
        # print(img_id['length'], len(str(img_id['text'])))
        y_label = (int(img_id['length']), torch.stack(list((img_id['text'][0]))), torch.stack(list((img_id['text'][1]))), torch.stack(list((img_id['text'][2]))))# (img_id['text'][3]), (img_id['text'][4]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [8]:
transform = transforms.Compose(
    [
        transforms.Resize((356, 356)),
        transforms.RandomCrop((299, 299)),
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, )),
    ]
)

In [9]:
BATCH = 16

In [10]:
dataset = CaptchaDataset(BASE_DIR, "Alpha.csv", transform = transform)
print(len(dataset))
train_set, validation_set = torch.utils.data.random_split(dataset, [1800,200])

train_loader = DataLoader(dataset = train_set, shuffle = True, batch_size = BATCH)
validation_loader = DataLoader(dataset = validation_set, shuffle = True, batch_size = BATCH)

2000


In [11]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(type(labels[1]))

TypeError: ignored

In [12]:
print(type(labels))

NameError: ignored

In [13]:
print(labels[1])

NameError: ignored

In [ ]:
DROPOUT_PROB = 0.25
N_EPOCHS = 10
# model_path = './Models/'

In [ ]:
class Model(nn.Module):
    def __init__(self, dropout_flag=0):
        super().__init__()

        # self.dropout_flag = dropout_flag  # 1->Conv Layer, 2->FC Layer

        self.blockA = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
        )

        # self.blockA_drop = nn.Sequential(
        #     nn.Conv2d(in_channels=3, out_channels=4, kernel_size=9, stride=1, padding=0),
        #     nn.ReLU(),
        #     nn.Dropout(DROPOUT_PROB),
        # )

        self.blockB = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
        )

        # self.blockB_drop = nn.Sequential(
        #     nn.Conv2d(in_channels=4, out_channels=6, kernel_size=6, stride=1, padding=0),
        #     nn.ReLU(),
        #     nn.Dropout(DROPOUT_PROB),
        # )

        self.blockC = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=6, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
        )

        # self.blockC_drop = nn.Sequential(
        #     nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=0),
        #     nn.ReLU(),
        #     nn.Dropout(DROPOUT_PROB),
        # )


        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(5400, 512)
        self.out1 = nn.Linear(512, 26)
        self.out2 = nn.Linear(512, 26)
        self.out3 = nn.Linear(512, 26)
        self.out4 = nn.Linear(512, 26)
        self.out5 = nn.Linear(512, 26)

        self.softmax = nn.Softmax()

        # self.drop = nn.Dropout(DROPOUT_PROB)
        

    def forward(self, x):
        
        # if (self.dropout_flag==0):
        x = self.blockA(x)
        x = self.pool(x)
        x = self.blockB(x)
        x = self.pool(x)
        x = self.blockC(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        # print(x.shape)
        x = self.fc1(x)
        label1 = self.softmax(self.out1(x))
        label2 = self.softmax(self.out2(x))
        label3 = self.softmax(self.out3(x))
        # label4 = self.softmax(self.out4(x))
        # label5 = self.softmax(self.out5(x))

        return {'label1': label1, 'label2': label2, 'label3': label3} #, 'label4': label4, 'label5': label5}

In [ ]:
import tensorflow as tf

def train(model, title, regularization = None):
    
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    trainlosslist = []
    validationlosslist = []

    for epoch in tqdm(range(N_EPOCHS)):
        train_loss = 0.0
        train_correct1 = 0
        train_correct2 = 0
        train_correct3 = 0
        train_correct4 = 0
        train_correct5 = 0
        train_total = 0

        validation_loss = 0.0
        validation_correct1 = 0
        validation_correct2 = 0
        validation_correct3 = 0
        validation_correct4 = 0
        validation_correct5 = 0
        validation_total = 0

        for i, data in enumerate(train_loader):
            model.train()
            inputs = data[0].to(device)
            # labels = tf. convert_to_tensor(data[1]).to(device)
            labels = data[1]
            label1 = labels[1].to(device)
            label2 = labels[2].to(device)
            label3 = labels[3].to(device)
            # label4 = labels[4].to(device)
            # label5 = labels[5].to(device)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.to(device))

            label1_hat = outputs['label1']
            label2_hat = outputs['label2']
            label3_hat = outputs['label3']         
            # label4_hat = outputs['label4']         
            # label5_hat = outputs['label5']  

            # calculate loss
            loss1 = criterion(label1_hat, label1.to(device).squeeze().type(torch.LongTensor))
            loss2 = criterion(label2_hat, label2.to(device).squeeze().type(torch.LongTensor))
            loss3 = criterion(label3_hat, label3.to(device).squeeze().type(torch.LongTensor))     
            # loss4 = criterion(label4_hat, label4.to(device).squeeze().type(torch.LongTensor))     
            # loss5 = criterion(label5_hat, label5.to(device).squeeze().type(torch.LongTensor))

            loss = loss1 + loss2 + loss3 #+ loss4 + loss5

            _, predicted1 = torch.max(label1_hat.data, 1)
            _, predicted2 = torch.max(label2_hat.data, 1)
            _, predicted3 = torch.max(label3_hat.data, 1)
            # _, predicted4 = torch.max(label4_hat.data, 1)
            # _, predicted5 = torch.max(label5_hat.data, 1)

            train_total += label1.size(0)
            
            train_correct1 += (predicted1 == label1.to(device)).sum().item()
            train_correct2 += (predicted2 == label2.to(device)).sum().item()
            train_correct3 += (predicted3 == label3.to(device)).sum().item()
            # train_correct4 += (predicted4 == label4.to(device)).sum().item()
            # train_correct5 += (predicted5 == label5.to(device)).sum().item()

            loss.backward()
            optimizer.step()

            # print statistics
            train_loss += loss.item()
            
            
        # for i, data in enumerate(validation_loader):
        #     model.eval()
        #     inputs, labels = data[0].to(device), data[1].to(device)
        #     labels = labels.to(torch.float32)
            
        #     outputs = model(inputs.to(device))
        #     outputs = torch.reshape(outputs, (outputs.shape[0],))
        #     loss = criterion(outputs, labels.to(device))
        #     validation_loss += loss.item()
            
            
        #     predicted = (outputs>0.5).float()
        #     validation_total += labels.size(0)
        #     validation_correct1 += (predicted == labels.to(device)).sum().item()
        #     validation_correct2 += (predicted == labels.to(device)).sum().item()
        #     validation_correct3 += (predicted == labels.to(device)).sum().item()
        #     validation_correct4 += (predicted == labels.to(device)).sum().item()
        #     validation_correct5 += (predicted == labels.to(device)).sum().item()


        epoch_train_loss = train_loss/len(train_loader)
        # epoch_validation_loss = validation_loss/len(validation_loader) 
        epoch_train_acc1 = (train_correct1/train_total)*100
        epoch_train_acc2 = (train_correct2/train_total)*100
        epoch_train_acc3 = (train_correct3/train_total)*100
        # epoch_train_acc4 = (train_correct4/train_total)*100
        # epoch_train_acc5 = (train_correct5/train_total)*100
        # epoch_validation_acc = (validation_correct1/validation_total)*100

        # epoch_train_acc2 = 0
        # epoch_train_acc3 = 0
        epoch_train_acc4 = 0
        epoch_train_acc5 = 0

        print("Epoch [%d/%d]  Training Loss: %.5f  Acc1: %.5f  Acc2: %.5f  Acc3: %.5f  Acc4: %.5f  Acc5: %.5f " %(epoch+1, N_EPOCHS, epoch_train_loss, epoch_train_acc1, epoch_train_acc2, epoch_train_acc3, epoch_train_acc4, epoch_train_acc5))
        
        # print("Epoch [%d/%d]  Training Loss: %.5f  Training Acc: %.5f  Validation Loss: %.5f   Validation Accuracy: %.5f " %(epoch+1, N_EPOCHS, epoch_train_loss, epoch_train_acc, epoch_validation_loss, epoch_validation_acc))
        trainlosslist.append(epoch_train_loss)
        # validationlosslist.append(epoch_validation_loss)
        
        # if (epoch_validation_loss < min_validation_loss):
        #     min_validation_loss = epoch_validation_loss
        #     torch.save(model.state_dict(), str(model_path)+str(title)+'.pt')
        

    print('Finished Training')
    
    # plot(trainlosslist, validationlosslist, title)

In [ ]:
model = Model().to(device)

In [ ]:
title = 'yeah'
train(model, title)

  0%|          | 0/10 [00:03<?, ?it/s]


AttributeError: ignored

In [ ]:
# def load_data(path, C_type='/AlphaNum', char_len = 5):
#     print('loading dataset')
#     X_train = []
#     Y_train = []
#     X_test = []
#     Y_test = []

#     path = path + C_type

#     for root, dirs, files in os.walk(path):
#         for f in files:
#             if 'Len5Char' in os.path.join(root,f):
#                 if '.png' in f:
#                     fl = f.split("_")[0]
#                     print(fl)
                    

#                     label = np.zeros((char_len, 36))
#                     for i in range(char_len):
#                         label[i, int(fl[i])] = 1

#                     img = cv2.imread(os.path.join(root, fl))
#                     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                     img = cv2.resize(img, (int(135/2), int(50/2)), interpolation=cv2.INTER_AREA)
#                     img = np.reshape(img, (img.shape[0], img.shape[1], 1))

#                     if random() < 0.1:
#                         Y_test.append(label)
#                         X_test.append(img)
#                     else:
#                         Y_train.append(label)
#                         X_train.append(img)






## Visualize Images

In [ ]:
def imshow(img, title):
    plt.figure()
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1,2,0))
    plt.title(title)
    plt.imshow(npimg)
    plt.show()

dataiter = iter(dataloader)
images, labels = dataiter.next()

c=0
for i in range(len(images)):
    if (labels[i]==0):
        c+=1
        imshow(images[i], "Parasitized")
    if (c==5):
        break

c=0
for i in range(len(images)):
    if (labels[i]==1):
        c+=1
        imshow(images[i], "Uninfected")
    if (c==5):
        break

In [ ]:
def plot(trainlosslist, validationlosslist, title):
    epochs = range(1,N_EPOCHS+1)
    plt.plot(epochs, trainlosslist, 'g', label='Training loss')
    plt.plot(epochs, validationlosslist, 'b', label='Validation loss')
    plt.title('Training and Validation loss  - ' + str(title))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def testing(model):
    
    model.eval()
    
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images.to(device))
            outputs = torch.reshape(outputs, (outputs.shape[0],))
            predicted = (outputs>0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels.to(device)).sum().item()
            

    print('Accuracy of the network on the test images: %.5f %%' % (100*test_correct/test_total))